In [20]:
from utils import *
warnings.filterwarnings('ignore')
%matplotlib inline


In [21]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [22]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")


In [23]:
from tkinter.filedialog import askopenfilename
import tkinter
from torchvision.io import read_image


## !feature extraction for full database!

In [19]:
start = 0
stop = len(dataset)
step = 2  # even-numbered image IDs only

for idx in range(start, stop, step):
    image_fd = get_all_fd(idx)

    # Store to collection (update if existing)
    fd_collection.update_one(
        {"image_id": idx},
        {"$set": image_fd},
        upsert=True,
    )


(3, 337, 510)
(3, 325, 492)
(3, 349, 528)
(3, 360, 545)
(3, 333, 504)
(3, 316, 479)
(3, 308, 466)
(3, 358, 542)
(3, 311, 471)
(3, 341, 516)
(3, 374, 566)
(3, 325, 492)
(3, 333, 504)
(3, 308, 466)
(3, 329, 498)
(3, 311, 471)
(3, 304, 461)
(3, 323, 489)
(3, 309, 468)
(3, 309, 468)
(3, 313, 473)
(3, 339, 513)
(3, 321, 487)
(3, 362, 548)
(3, 349, 528)
(3, 362, 548)
(3, 367, 555)
(3, 331, 501)
(3, 347, 525)
(3, 318, 481)
(3, 327, 495)
(3, 360, 545)
(3, 333, 504)
(3, 351, 532)
(3, 304, 461)
(3, 313, 473)
(3, 308, 466)
(3, 295, 447)
(3, 301, 456)
(3, 313, 473)
(3, 304, 461)
(3, 303, 458)
(3, 308, 466)
(3, 318, 481)
(3, 320, 484)
(3, 418, 633)
(3, 395, 598)
(3, 424, 642)
(3, 389, 589)
(3, 412, 623)
(3, 430, 652)
(3, 384, 582)
(3, 412, 623)
(3, 409, 619)
(3, 379, 574)
(3, 418, 633)
(3, 339, 513)
(3, 295, 447)
(3, 289, 437)
(3, 349, 528)
(3, 367, 555)
(3, 313, 473)
(3, 339, 513)
(3, 276, 418)
(3, 314, 476)
(3, 335, 507)
(3, 337, 510)
(3, 333, 504)
(3, 320, 484)
(3, 325, 492)
(3, 341, 516)
(3, 31

OperationFailure: you are over your space quota, using 514 MB of 512 MB, full error: {'ok': 0, 'errmsg': 'you are over your space quota, using 514 MB of 512 MB', 'code': 8000, 'codeName': 'AtlasError'}

### Get input as image ID (dataset) or from image file

In [24]:
selected_image_id = int(
    input("Enter image ID: (-1 if you want to select an image file)")
)
if selected_image_id == -1:
    # pick image and get label using tkinter
    root = tkinter.Tk()
    root.attributes("-topmost", True)
    selected_image_path = askopenfilename(title="Select Image File")
    root.withdraw()
    print(selected_image_path)
    sample_image = read_image(selected_image_path).div(255)  # scale to [0,1] range
    sample_label = int(input("Enter image label number: (0 to 100)"))
else:
    # get image and label from dataset
    sample_image, sample_label = dataset[selected_image_id]

# Show selected image
plt.imshow(transforms.ToPILImage()(sample_image))
plt.show()

k = int(input("Enter value of k: "))
if k < 1:
    raise ValueError("k should be a positive integer")

selected_feature_model = valid_feature_models[
    str(input("Enter feature model - one of " + str(list(valid_feature_models.keys()))))
]

selected_distance_measure = valid_distance_measures[
    str(input("Enter distance measure - one of " + str(list(valid_distance_measures.keys()))))
]

if selected_image_id == -1:
    show_similar_images_for_image(
        fd_collection,
        -1,
        sample_image,
        sample_label,
        k,
        selected_feature_model,
        selected_distance_measure,
        save_plots=False,
    )
else:
    show_similar_images_for_image(
        fd_collection,
        selected_image_id,
        None,
        None,
        k,
        selected_feature_model,
        selected_distance_measure,
        save_plots=False,
    )


ValueError: invalid literal for int() with base 10: ''